In [ ]:
!nvidia-smi

Sun Jun 14 11:07:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.36.06    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorboardx
!mkdir checkpoints
!mkdir results


     |████████████████████████████████| 204kB 2.8MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import BatchSampler
from itertools import cycle
from tensorboardX import SummaryWriter
import torch.autograd as autograd
from tqdm import tqdm
import matplotlib.pyplot as plt


In [ ]:

class BalancedBatchSampler(BatchSampler):
    """
    BatchSampler - from a MNIST-like dataset, samples n_classes and within these classes samples n_samples.
    Returns batches of size n_classes * n_samples
    """

    def __init__(self, dataset, n_classes, n_samples,custom_classes =None):
        loader = DataLoader(dataset)
        self.labels_list = []
        self.custom_classes = custom_classes
        for _, label in loader:
            self.labels_list.append(label)
        self.labels = torch.LongTensor(self.labels_list)
        self.labels_set = list(set(self.labels.numpy()))
        self.label_to_indices = {label: np.where(self.labels.numpy() == label)[0]
                                 for label in self.labels_set}
        for l in self.labels_set:
            np.random.shuffle(self.label_to_indices[l])
        self.used_label_indices_count = {label: 0 for label in self.labels_set}

        self.count = 0
        self.n_classes = n_classes
        self.n_samples = n_samples
        self.dataset = dataset
        self.batch_size = self.n_samples * self.n_classes

    def __iter__(self):
        self.count = 0
        while self.count + self.batch_size < len(self.dataset):
            classes = np.random.choice(self.labels_set, self.n_classes, replace=False)
            indices = []
            if self.custom_classes!=None:
                classes = self.custom_classes
            for class_ in classes:
                indices.extend(self.label_to_indices[class_][
                               self.used_label_indices_count[class_]:self.used_label_indices_count[
                                                                         class_] + self.n_samples])
                self.used_label_indices_count[class_] += self.n_samples
                if self.used_label_indices_count[class_] + self.n_samples > len(self.label_to_indices[class_]):
                    np.random.shuffle(self.label_to_indices[class_])
                    self.used_label_indices_count[class_] = 0
            yield indices
            self.count += self.n_classes * self.n_samples

    def __len__(self):
        return len(self.dataset) // self.batch_size

In [ ]:


writer = SummaryWriter('runs_gan/correctedmetaclassifier_wgan_mnist')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


img_size = 28
##Simulataneous training of different classes
n_classes = 1
#custom_classes = np.random.randint(0,10,n_classes)
custom_classes = np.array([9])
#custom_classes = None
class_batch = 64
transform = transforms.Compose([torchvision.transforms.Resize(img_size),torchvision.transforms.ToTensor()])
dataset = torchvision.datasets.MNIST(root="./", train=True,download=True,transform=transform)
#balanced_batch_sampler = BalancedBatchSampler(dataset, n_classes, class_batch,custom_classes)
batch_size = 64

#dataloader = torch.utils.data.DataLoader(dataset, batch_sampler=balanced_batch_sampler)
class_datasets = [torch.utils.data.dataset.Subset(dataset, np.where(dataset.targets==i)[0]) for i in range(10)]
dataloaders = [torch.utils.data.DataLoader(class_datasets[i],batch_size=batch_size,shuffle=True,num_workers=4) for i in range(10)]
##parameters

dim = 64
LAMBDA = 10
epochs= 20000
n_z = 128
k = 10

In [ ]:


class netD(nn.Module):
    def __init__(self):
        super(netD,self).__init__()
        self.conv1 = nn.Conv2d(1,dim,5,stride=2,padding=2)
        self.act1 = nn.ReLU(True)
        self.conv2 = nn.Conv2d(dim,dim*2,5,stride=2,padding=2)
        self.act2 = nn.ReLU(True)
        self.conv3 = nn.Conv2d(dim*2,dim*4,5,stride=2,padding=2)
        self.act3 = nn.ReLU(True)
        self.fc1 = nn.Linear(4*4*4*dim,1)

    def forward(self,input):
        input = input.view(-1,1,28,28)

        l1_1 = self.conv1(input)
        l1_2 = self.act1(l1_1)
        l2_1 = self.conv2(l1_2)
        l2_2 = self.act2(l2_1)
        l3_1 = self.conv3(l2_2)
        l3_2 = self.act3(l3_1)

        l4_1 = self.fc1(l3_2.view(-1,4*4*4*dim))
        return l4_1.view(-1)


class netG(nn.Module):
    def __init__(self):
        super(netG,self).__init__()
        self.convt1 = nn.Linear(128,4*4*4*dim)
        self.act1 = nn.ReLU(True)
        self.convt2 = nn.ConvTranspose2d(4*dim,dim*2,5)
        self.act2 = nn.ReLU(True)
        self.convt3 = nn.ConvTranspose2d(dim*2,dim,5)
        self.act3 = nn.ReLU(True)
        self.convt4 = nn.ConvTranspose2d(dim,1,8,stride=2)
        self.sigmoid = nn.Sigmoid()

    def forward(self,input):
        #input = input.view(-1,1,28,28)
        l1_1 = self.convt1(input)
        l1_2 = self.act1(l1_1)
        l1_2 = l1_2.view(-1,4*dim,4,4)
        l2_1 = self.convt2(l1_2)
        l2_2 = self.act2(l2_1)
        l2_2 = l2_2[:,:,:7,:7]
        l3_1 = self.convt3(l2_2)
        l3_2 = self.act3(l3_1)
        l4_1 = self.convt4(l3_2)
        output = self.sigmoid(l4_1)
        return output.view(-1,img_size*img_size)


In [ ]:

fixed_batch = torch.rand(class_batch,128,device=device,requires_grad=False)

def calc_gradient_penalty(netD, real_data, fake_data):
    #print real_data.size()
    alpha = torch.rand(class_batch, 1).to(device)
    alpha = alpha.expand(real_data.size())
    interpolates = alpha * real_data + ((1 - alpha) * fake_data)

    interpolates = autograd.Variable(interpolates, requires_grad=True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty
#criterion = nn.BCEWithLogitsLoss()
#metanet = netD().to(device)
#tasknet = netD().to(device)
#taskoptim = optim.Adam(tasknet.parameters(),lr=0.0001)
#metaoptim = optim.Adam(metanet.parameters(),lr=0.00001)

##training

metadiscriminator = netD().to(device)
discriminator = netD().to(device)
optimizerD = optim.Adam(discriminator.parameters(), lr=1e-4, betas=(0.5, 0.9))
metaoptimizerD = optim.Adam(metadiscriminator.parameters(), lr=1e-5)


In [ ]:
discriminator.load_state_dict(torch.load("./checkpoints/run2_gan_discriminator_180009"))

FileNotFoundError: ignored

In [ ]:

iters=0
for epoch in range(epochs):
  index1 = np.random.randint(0,10)
  index2 = np.random.randint(0,10)
  while index1==index2:
    index2 = np.random.randint(0,10)
  
  data_iterator = iter(dataloaders[index1])
  fake_iterator = iter(dataloaders[index2])
  index = 0
  metaoptimizerD.zero_grad()
  discriminator.load_state_dict(metadiscriminator.state_dict())
  for _ in range(k):
    #print("cola")
    discriminator.zero_grad()
    x_real = data_iterator.next()[0].to(device)
    x_real = x_real.view(-1,img_size*img_size)
    b_size = x_real.shape[0]
    output = discriminator(x_real).view(-1)
    D_real = output.mean()

    x_fake = fake_iterator.next()[0].to(device)
    fake_data = x_real.view(-1,img_size*img_size)
    output = discriminator(fake_data).view(-1)
    D_fake = output.mean()
    D_loss = D_fake - D_real
    D_loss.backward()
    gradient_p = calc_gradient_penalty(discriminator,x_real.detach(),fake_data.detach())
    gradient_p.backward()
    total_D_loss = D_loss + gradient_p
    optimizerD.step()
    writer.add_scalar('Discriminator_Loss', total_D_loss,iters) 
  for p,metap in zip(discriminator.parameters(),metadiscriminator.parameters()):
    diff = metap-p
    metap.grad = diff
  metaoptimizerD.step()
  if iters%200==0:
    print("Epoch : "+str(epoch)+" D_Loss: " + str(total_D_loss.item()))
  if epoch%1000==0:
    torch.save(discriminator.state_dict(),'./checkpoints/run4_gan_discriminator_'+str(iters))
  iters = iters+1





 






In [ ]:

!zip -r correctedmetaclassifiermnist_result.zip ./results/run*
!zip -r correctedmetaclassifiermnist_checkpoint.zip ./checkpoints/run4*
!zip -r correctedmetaclassifiermnist_log.zip ./runs_gan

	zip warning: name not matched: ./results/run*

zip error: Nothing to do! (try: zip -r correctedmetaclassifiermnist_result.zip . -i ./results/run*)
  adding: checkpoints/run4_gan_discriminator_0 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_1000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_10000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_11000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_12000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_13000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_14000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_15000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_16000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_17000 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_18296 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_19296 (deflated 8%)
  adding: checkpoints/run4_gan_discriminator_2000 (deflated 8%)
  adding: che

In [ ]:
!rm -r ./checkpoints/*
!rm -r ./results/*
!rm -r ./runs_gan/*

In [ ]:
!mv correctedmetaclassifiermnist_result.zip /content/drive/"My Drive"/
!mv correctedmetaclassifiermnist_checkpoint.zip /content/drive/"My Drive"/
!mv correctedmetaclassifiermnist_log.zip /content/drive/"My Drive"/



mv: cannot stat 'correctedmetaclassifiermnist_result.zip': No such file or directory
